In [5]:
!pip install git+https://github.com/openai/CLIP.git
!pip install torch torchvision
!pip install pillow tqdm

  Cloning https://github.com/openai/CLIP.git to c:\users\user\appdata\local\temp\pip-req-build-izz_qhii
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   --------- ------------------------------ 10.2/44.8 kB ? eta -:--:--
   ------------------------------------ --- 41.0/44.8 kB 495.5 kB/s eta 0:00:01
   ---------------------------------------- 44.8/44.8 kB 443.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.2/1.6 MB 7.0 MB/s eta 0:00:01
   ----------------------- ---------------- 0.9/1.6 MB 11.7 MB/s eta 0:00:01
   ---------------------------------- ----- 1.4/1.6 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 9.0 MB/s eta 0:00:01
   ------------------

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-izz_qhii'


In [6]:
import torch
import clip
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import json
import os
from tqdm import tqdm

In [12]:
# JSON 데이터 로드

json_foler_path = r"E:\AI_HUB\data\Sublabel\SbL\extracted_data.json"
with open(json_foler_path, 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

In [7]:
# 이미지 데이터셋 클래스 정의
class TourismDataset(Dataset):
    def __init__(self, data, image_dir, preprocess):
        self.data = data
        self.image_dir = image_dir
        self.preprocess = preprocess

        # 이미지와 텍스트 매칭 데이터 생성
        self.samples = [
            (os.path.join(image_dir, entry["PHOTO_FILE_NM"]), " ".join(entry["FILTERED_KEYWORDS"]))
            for entry in data
            if os.path.exists(os.path.join(image_dir, entry["PHOTO_FILE_NM"]))
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, keywords = self.samples[idx]
        image = self.preprocess(Image.open(image_path).convert("RGB"))
        return image, keywords

In [8]:
# CLIP 모델과 Preprocessor 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:23<00:00, 15.2MiB/s]


In [9]:
# 이미지 디렉토리 경로 설정
image_directory = r"E:\AI_HUB\data\Sublabel\SbL\photo"  

In [13]:
# 데이터셋 생성 및 DataLoader 정의
dataset = TourismDataset(data, image_directory, preprocess)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [15]:
# 학습 루프
for images, texts in tqdm(dataloader):
    # 이미지를 CLIP 모델 입력 형식으로 변환
    images = images.to(device)
    texts = clip.tokenize(texts).to(device)

    # CLIP 모델로 피처 추출
    with torch.no_grad():
        image_features = model.encode_image(images)
        text_features = model.encode_text(texts)

    # 이미지와 텍스트 피처 정규화
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # 코사인 유사도 계산
    similarity = (image_features @ text_features.T).cpu().numpy()

    # 유사도 확인 (예: 학습 진행 상황 모니터링)
    print("유사도 매트릭스:", similarity)

  0%|          | 0/192 [00:00<?, ?it/s]C:\Users\user\anaconda3\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
  1%|          | 1/192 [00:12<40:12, 12.63s/it]

유사도 매트릭스: [[0.23195186 0.2346167  0.21427679 0.22538793 0.23596917 0.23591122
  0.22311148 0.22935568 0.24648868 0.21809337 0.22680146 0.2254886
  0.20428559 0.21809337 0.2344031  0.20825118]
 [0.2038758  0.20155872 0.18247567 0.17920695 0.20560189 0.19478543
  0.18986052 0.19618595 0.21860659 0.1909531  0.18763137 0.19350031
  0.17860161 0.1909531  0.19903341 0.17881984]
 [0.23750636 0.24376312 0.22106987 0.22146818 0.24413438 0.23927799
  0.23178452 0.23789757 0.25374514 0.22964945 0.23529398 0.23656495
  0.21846414 0.22964945 0.24178186 0.22573557]
 [0.23710635 0.240179   0.2130803  0.22584979 0.2501044  0.2433073
  0.22670373 0.22999379 0.24205793 0.22355765 0.22585592 0.22870433
  0.2144044  0.22355765 0.24315661 0.21934226]
 [0.25924096 0.24939235 0.23739903 0.23369806 0.25852877 0.25526726
  0.24271724 0.2519938  0.268462   0.24028254 0.24408866 0.24607456
  0.23127201 0.24028254 0.24379236 0.23625778]
 [0.21345222 0.21743181 0.19474095 0.21571322 0.2210589  0.2221525
  0.214684

  1%|          | 2/192 [00:21<32:38, 10.31s/it]

유사도 매트릭스: [[0.2379782  0.2167168  0.21650451 0.24083284 0.21779412 0.21174562
  0.2140156  0.21883437 0.20260584 0.20713143 0.21757826 0.2194019
  0.20994344 0.21126235 0.21085683 0.21634504]
 [0.2071555  0.18037874 0.18088762 0.20886953 0.19407022 0.1670902
  0.19219984 0.18625635 0.17244846 0.18003006 0.17635335 0.18055122
  0.17658332 0.17916384 0.18251628 0.18917648]
 [0.20341161 0.1878156  0.17403747 0.20635438 0.18913358 0.17490432
  0.16661772 0.19251607 0.17595059 0.18927509 0.18251947 0.17230017
  0.18564737 0.18577528 0.17931649 0.18471722]
 [0.21330336 0.19606908 0.1893585  0.21581942 0.20612726 0.18785077
  0.18950106 0.19356826 0.18901666 0.18598765 0.19381957 0.19196817
  0.18901429 0.19281968 0.18684036 0.18578449]
 [0.26143464 0.22578238 0.21942902 0.2647969  0.23217316 0.21389931
  0.20970047 0.23839793 0.21464348 0.22485271 0.22208959 0.22145244
  0.22546062 0.22612253 0.21914914 0.23297577]
 [0.24421707 0.22423777 0.22562432 0.24847156 0.23146531 0.2142986
  0.227999

  2%|▏         | 3/192 [00:32<33:24, 10.60s/it]

...

유사도 매트릭스: [[0.20806639 0.21792802 0.21473128 0.21057814 0.24265659 0.21954694
  0.19848171 0.20301837 0.2338341  0.20163132 0.23320007 0.24712156
  0.22423917 0.19984034 0.21257927 0.20224616]
 [0.1998848  0.20098765 0.20525004 0.2000185  0.23087057 0.20189953
  0.18773131 0.19729039 0.22513688 0.19729842 0.21336026 0.23099832
  0.2080757  0.18989688 0.20522061 0.18898916]
 [0.20610425 0.22440018 0.21297938 0.21154183 0.24630253 0.21532933
  0.19681329 0.1965211  0.24173087 0.1960265  0.23330969 0.24945167
  0.23484805 0.19528267 0.21696387 0.20137334]
 [0.22936907 0.23672602 0.24269198 0.23062564 0.25275725 0.23725379
  0.21852382 0.21288553 0.24651061 0.22944158 0.23455659 0.2569343
  0.24644963 0.2196924  0.23159012 0.22154933]
 [0.21239778 0.22552758 0.22175238 0.21632493 0.23366289 0.2066237
  0.20105413 0.19193088 0.23511809 0.21071771 0.21478495 0.23327318
  0.23063043 0.2056245  0.20368685 0.20439515]
 [0.20204604 0.2165083  0.2133808  0.214333   0.24191168 0.21107797
  0.20057

 99%|█████████▉| 191/192 [32:34<00:08,  8.89s/it]

유사도 매트릭스: [[0.20556754 0.20165715 0.20194343 0.22711891 0.19718689 0.19228294
  0.19451834 0.2058092  0.21219552 0.20662454 0.19967403 0.20189783
  0.19304025 0.20336023 0.20085657 0.18853468]
 [0.22669032 0.22930667 0.23840636 0.24805635 0.21796942 0.22800419
  0.2126965  0.24287274 0.23208684 0.22925565 0.23587593 0.22809058
  0.21827155 0.21827811 0.2310678  0.21426375]
 [0.20564784 0.21079805 0.22229666 0.25411505 0.19520858 0.1973418
  0.20036171 0.22401035 0.21817385 0.20379983 0.21336117 0.20408008
  0.19427893 0.19277215 0.20474851 0.19062197]
 [0.20522575 0.201554   0.2250483  0.2556835  0.18415523 0.19239196
  0.19323623 0.21715802 0.2101293  0.19751841 0.21649164 0.20506302
  0.1846258  0.18555178 0.20245785 0.18145704]
 [0.21545276 0.21812317 0.21745825 0.24260154 0.21458304 0.21616581
  0.21234986 0.23163661 0.22587661 0.22163725 0.21631674 0.22092691
  0.21355867 0.21526259 0.20985922 0.20926324]
 [0.21533498 0.2098796  0.209797   0.23446006 0.20835751 0.21230185
  0.2062

100%|██████████| 192/192 [32:40<00:00, 10.21s/it]

유사도 매트릭스: [[0.20288114 0.2156761  0.20029658 0.18990205 0.19916795 0.20069565
  0.20875593 0.20318961 0.1903083  0.1950749 ]
 [0.20326455 0.22110751 0.20596822 0.19061284 0.20032129 0.1984885
  0.21575737 0.20571862 0.19275099 0.18959917]
 [0.23155862 0.26008838 0.23007686 0.21383163 0.22522524 0.22271651
  0.24289192 0.2319494  0.2155543  0.22784191]
 [0.19862553 0.21890742 0.19507179 0.18107843 0.19316372 0.19001764
  0.21023962 0.19788547 0.1815135  0.18563668]
 [0.18233438 0.2021654  0.17533618 0.16177344 0.17642531 0.17208411
  0.18557768 0.18139444 0.1637567  0.17704453]
 [0.18177047 0.18567416 0.17359741 0.15490086 0.17453562 0.1717827
  0.17361695 0.17989731 0.15454186 0.16638458]
 [0.22073042 0.24408485 0.21587095 0.20354795 0.2137132  0.2151606
  0.22909951 0.22145352 0.20688005 0.20878261]
 [0.16976213 0.18615146 0.1551489  0.14084774 0.15850905 0.15580344
  0.16469482 0.16377757 0.14318737 0.14644933]
 [0.19788013 0.21278149 0.19601119 0.18704073 0.19452149 0.19005805
  0.2

In [18]:
def precision_at_k(similarity_matrix, k=5):
    correct_count = 0
    for i in range(similarity_matrix.shape[0]):
        top_k_indices = similarity_matrix[i].argsort()[-k:][::-1]
        if i in top_k_indices:
            correct_count += 1
    return correct_count / similarity_matrix.shape[0]

# Precision@5 계산
precision = precision_at_k(similarity, k=5)
print(f"Precision@5: {precision:.2f}")

Precision@5: 0.50
